In [1]:
%matplotlib inline

import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import time
from sklearn.externals import joblib
import sklearn_recommender as skr

/home/bruce/.local/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
[nltk_data] Downloading package punkt to /home/bruce/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/bruce/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/bruce/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/bruce/nltk_data...
[nltk_data]   Package wordnet is already up-to

In [2]:
triplets_file = '/home/bruce/MusicRec/10000.txt'
songs_metadata_file = '/home/bruce/MusicRec/song_data.csv'

song_df_1 = pd.read_table(triplets_file,header=None)

song_df_1.columns = ['user_id', 'song_id', 'listen_count']

song_df_2 =  pd.read_csv(songs_metadata_file)

#Merge the two dataframes above to create input dataframe for recommender systems
song_df = pd.merge(song_df_1, song_df_2.drop_duplicates(['song_id']), on="song_id", how="left")

In [3]:
song_df.head()

,user_id,song_id,listen_count,title,release,artist_name,year
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,The Cove,Thicker Than Water,Jack Johnson,0
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,Stronger,Graduation,Kanye West,2007
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1,Constellations,In Between Dreams,Jack Johnson,2005
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1,Learn To Fly,There Is Nothing Left To Lose,Foo Fighters,1999


In [4]:
song_grouped = song_df.groupby(['title']).agg({'listen_count': 'count'}).reset_index()
grouped_sum = song_grouped['listen_count'].sum()
song_grouped['percentage']  = song_grouped['listen_count'].div(grouped_sum)*100
song_grouped.sort_values(['listen_count', 'title'], ascending = [0,1])


,title,listen_count,percentage
6836,Sehr kosmisch,8277,0.41385
8725,Undo,7032,0.35160
1964,Dog Days Are Over (Radio Edit),6949,0.34745
9496,You're The One,6729,0.33645
6498,Revelry,6145,0.30725
...,...,...,...
3363,Historia Del Portero,51,0.00255
6782,Scared,51,0.00255
2041,Don´t Leave Me Now,50,0.00250
2860,Ghosts (Toxic Avenger Mix),48,0.00240


In [5]:
users = song_df['user_id'].unique()
users

array(['b80344d063b5ccb3212f76538f3d9e43d87dca9e',
       '85c1f87fea955d09b4bec2e36aee110927aedf9a',
       'bd4c6e843f00bd476847fb75c47b4fb430a06856', ...,
       'a955513fb89fdb0d5e8437a5cf8a9b3a0abad4d5',
       '276e43ad698705e5011e5091f367d951b21246f5',
       'd8bfd4ec88f0f3773a9e022e3c1a0f1d3b7b6a92'], dtype=object)

In [6]:
songs = song_df['title'].unique()
songs

array(['The Cove', 'Entre Dos Aguas', 'Stronger', ..., 'Super Honeymoon',
       'Compañera',
       "Baby [You've Got What It Takes] [with Sharon Jones & the Dap-Kings]"],
      dtype=object)

In [7]:
ratings = pd.DataFrame(song_grouped.groupby('title')['percentage'].mean())
ratings['listen_count'] = pd.DataFrame(song_grouped.groupby('title')['listen_count'].mean())
ratings.head()

,percentage,listen_count
title,,
#!*@ You Tonight [Featuring R. Kelly] (Explicit Album Version),0.00390,78
#40,0.01690,338
& Down,0.01865,373
' Cello Song,0.00515,103
'97 Bonnie & Clyde,0.00465,93


In [11]:
song_df=song_df.head(10000)
mat = song_df.pivot_table(index='user_id',columns='title',values='listen_count')

In [16]:
Sehrkosmisch_ratings= mat['Sehr kosmisch']
similar_to_Sehrkosmisch_ratings = mat.corrwith(Sehrkosmisch_ratings)


/home/bruce/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:2526: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/home/bruce/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:2455: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


,Correlation
title,
'Till I Collapse,1.000000
A-Punk (Album),0.800000
Ain't Misbehavin,-0.208817
Ain't No Rest For The Wicked (Original Version),1.000000
Al otro lado del Río,1.000000
...,...
Yeah!,-0.866025
Yellow,0.588612
You Belong With Me,-0.126363


In [17]:
corr= pd.DataFrame(similar_to_Sehrkosmisch_ratings,columns=['Correlation'])
corr.dropna(inplace=True)
corr

,Correlation
title,
'Till I Collapse,1.000000
A-Punk (Album),0.800000
Ain't Misbehavin,-0.208817
Ain't No Rest For The Wicked (Original Version),1.000000
Al otro lado del Río,1.000000
...,...
Yeah!,-0.866025
Yellow,0.588612
You Belong With Me,-0.126363
